## (1) This dashboard for United flights analysis

In [3]:
## Wonderful!!!
#!pip install jupyter-dash
#!pip install jupyter-plotly

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


<ipython-input-3-b27af5c5d040>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-3-b27af5c5d040>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [ ]:
import pandas as pd


# Read the airline data into pandas dataframe
df =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})


In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


# Create a dash application
app = dash.Dash(__name__)

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})


# List of years 
year_list = [i for i in range(2005, 2021, 1)]

"""Compute graph data for creating yearly airline performance report 

Function that takes airline data as input and create 5 dataframes based on the grouping condition to be used for plottling charts and grphs.

Argument:
     
    df: Filtered dataframe
    
Returns:
   Dataframes to create graph. 
"""
def compute_data_choice_1(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    # Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data


"""Compute graph data for creating yearly airline delay report

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    df: Input airline data.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.
"""
def compute_data_choice_2(df):
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


# Application layout
app.layout = html.Div(children=[ 
                                # TASK1: Add title to the dashboard
                                # Enter your code below. Make sure you have correct formatting.
                                html.H1('US Domestic Airline Flight Performance', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
                                # REVIEW2: Dropdown creation
                                # Create an outer division 
                                html.Div([
                                    html.Div([
                                        html.H2('Report Type:', style={'margin-right': '2em'}),

                                        dcc.Dropdown(id='input-type',
                                            options=[
                                                {'label': 'Yearly Airline Performance Report', 'value': 'OPT1'},
                                                {'label': 'Yearly Airline Delay Report', 'value': 'OPT2'},
                                            ],
                                                placeholder='Select a report type',
                                                style={'width':'80%', 'padding':'3px','font-size':'20px',
                                                'textAlign': 'center'},
                                                )], style={'display':'flex'}),
                                    # Create an division for adding dropdown helper text for choosing year
                                    html.Div([
                                        html.H2('Choose Year:', style={'margin-right': '2em'}),

                                        # Update dropdown values using list comphrehension
                                        dcc.Dropdown(id='input-year', 
                                                    options=[{'label': i, 'value': i} for i in year_list],
                                                    placeholder="Select a year",
                                                    style={'width':'80%', 'padding':'3px', 'font-size': '20px', 
                                                    'text-align-last' : 'center'}),
                                            # Place them next to each other using the division style
                                            ], style={'display': 'flex'}),  
        
                                
                                    # Add Computed graphs
                                    # REVIEW3: Observe how we add an empty division and providing an id that will be updated during callback
                                    html.Div([  
                                        html.Div([ ], id='plot1'),
            
                                        html.Div([
                                                html.Div([ ], id='plot2'),
                                                html.Div([ ], id='plot3'),
                                        ], style={'display': 'flex'}),
                                        
                                        # TASK3: Add a division with two empty divisions inside. See above disvision for example.
                                        # Enter your code below. Make sure you have correct formatting.
                                        html.Div([
                                                html.Div([ ], id='plot4'),
                                                html.Div([ ], id='plot5'),
                                        ], style={'display': 'flex'}),
                                        ]),
                                    ]),
                            ])


# Callback function definition
# TASK4: Add 5 ouput components
# Enter your code below. Make sure you have correct formatting.
@app.callback(
    Output(component_id='plot1', component_property='children'),
    Output(component_id='plot2', component_property='children'),
    Output(component_id='plot3', component_property='children'),
    Output(component_id='plot4', component_property='children'),
    Output(component_id='plot5', component_property='children'),
    Input(component_id='input-type', component_property='value'),
    Input(component_id='input-year', component_property='value'),
    State("plot1", 'children'), 
    State("plot2", "children"),
    State("plot3", "children"), 
    State("plot4", "children"),
    State("plot5", "children"))
    # REVIEW4: Holding output state till user enters all the form information. In this case, 
    # it will be chart type and year
    
              
# Add computation to callback function and return graph
def get_graph(chart, year, children1, children2, c3, c4, c5):
      
        # Select data
        df =  airline_data[airline_data['Year']==int(year)]
       
        if chart == 'OPT1':
            # Compute required information for creating graph from the data
            bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df)
            
            # Number of flights under different cancellation categories
            bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
            
            # TASK5: Average flight time by reporting airline
            # Enter your code below. Make sure you have correct formatting.
            line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline', title='Averagne Monthly Flight time(minutes) by airline')
            
            
            # Percentage of diverted airport landings per reporting airline
            pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
            
            # REVIEW5: Number of flights flying from each state using choropleth
            map_fig = px.choropleth(map_data,  # Input data
                    locations='OriginState', 
                    color='Flights',  
                    hover_data=['OriginState', 'Flights'], 
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale='GnBu',
                    range_color=[0, map_data['Flights'].max()]) 
            map_fig.update_layout(
                    title_text = 'Number of flights from origin state', 
                    geo_scope='usa') # Plot only the USA instead of globe
            
            # TASK6: Number of flights flying to each state from each reporting airline
            # Enter your code below. Make sure you have correct formatting.
            tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'], 
                      values='Flights',
                      color='Flights',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state'
                )
            
            
            
            # REVIEW6: Return dcc.Graph component to the empty division
            return [dcc.Graph(figure=tree_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=map_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)]
        else:
            # REVIEW7: This covers chart type 2 and we have completed this exercise under Flight Delay Time Statistics Dashboard section
            # Compute required information for creating graph from the data
            avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
            
            # Create graph
            carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
            weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
            nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
            sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
            late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
            return [dcc.Graph(figure=carrier_fig), 
                   dcc.Graph(figure=weather_fig), 
                   dcc.Graph(figure=nas_fig), 
                   dcc.Graph(figure=sec_fig), 
                   dcc.Graph(figure=late_fig)]


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


## (2) Follow the skill network step by step --SpaceX Capstone

In [26]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import Input, Output, html, dcc
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 

                                            options=[
                                                {'label':'allSite' , 'value':'ALL' } ,
                                                {'label':'CCAFS LC-40' , 'value':'CCAFS LC-40' } ,
                                                {'label':'VAFB SLC-4E' , 'value':'VAFB SLC-4E' } ,
                                                {'label':'KSC LC-39A' , 'value':'KSC LC-39A' } ,
                                                {'label':'CCAFS SLC-40' , 'value':'CCAFS SLC-40' } ,
                                                    ],
                                                    value='ALL',
                                            placeholder="allSite",
                                            searchable=True,
                                            style={'width':'80%', 'padding':'3px', 'font-size': '20px', 
                                            'text-align-last' : 'center'},),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                        min=0, max=10000, step=1000,
                                        marks={0: '0', 100: '100'},
                                        value=[min_payload, max_payload]),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),  # output id respect the outlayer id.
    Input(component_id='site-dropdown', component_property='value'),
    )
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class',
        names='Launch Site', 
        title='title')
        return fig
    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]['class'].value_counts().reset_index(),
        fig = px.pie(df, values='class',
        names='class', 
        title='title')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider',component_property='value'),
)
def get_payload_scatter_chart(entered_site,slider_range):   # the paramter is combine the dropdown,and slider_range, how many input respect the paramter.
    filtered_df = spacex_df
    low, high = slider_range
    #mask = (df['petal_width'] > low) & (df['petal_width'] < high)
    #df = px.data.iris() # iris is a pandas DataFrame
    if entered_site == 'ALL':
        pldf=spacex_df.loc[(spacex_df['Payload Mass (kg)']<=high) & (spacex_df['Payload Mass (kg)']>=low)]
        df = px.data.pldf()
        fig= px.scatter(spacex_df,y='class',x='Payload Mass (kg)',color='Booster Version Category')
        return fig

    else:
        df=spacex_df.loc[spacex_df['Launch Site']==entered_site]
        pldf=df.loc[(df['Payload Mass (kg)']<=high) & (spacex_df['Payload Mass (kg)']>=low)]
        df = px.data.pldf()
        fig = px.catterie(df, y='class',x='Payload Mass (kg)', color='Booster Version Category',
        names='f{entered_site} Success Launch',
        title='title')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


##(2) Spacex dashboard capstone dataset analysis---this is the procedure!

In [4]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [29]:
# Read the airline data into pandas dataframe
#spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df = pd.read_csv('spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()
site_list=spacex_df['Launch Site'].unique().tolist()
site_list

,Unnamed: 0,Unnamed: 0.1,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,0,1,CCAFS LC-40,0,0.00,F9 v1.0 B0003,v1.0
1,1,1,2,CCAFS LC-40,0,0.00,F9 v1.0 B0004,v1.0
2,2,2,3,CCAFS LC-40,0,525.00,F9 v1.0 B0005,v1.0
3,3,3,4,CCAFS LC-40,0,500.00,F9 v1.0 B0006,v1.0
4,4,4,5,CCAFS LC-40,0,677.00,F9 v1.0 B0007,v1.0
5,5,5,7,CCAFS LC-40,0,3170.00,F9 v1.1,v1.1
6,6,6,8,CCAFS LC-40,0,3325.00,F9 v1.1,v1.1
7,7,7,9,CCAFS LC-40,0,2296.00,F9 v1.1,v1.1
8,8,8,10,CCAFS LC-40,0,1316.00,F9 v1.1,v1.1
9,9,9,11,CCAFS LC-40,0,4535.00,F9 v1.1,v1.1


In [27]:
spacex_df.to_csv('spacex_launch_dash.csv')

In [7]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [8]:
sbar_df=spacex_df.groupby(['Launch Site'])['class'].mean().reset_index()
sbar_df['failed']=1-(sbar_df['class'])
sbar_df

,Launch Site,class,failed
0,CCAFS LC-40,0.269231,0.730769
1,CCAFS SLC-40,0.428571,0.571429
2,KSC LC-39A,0.769231,0.230769
3,VAFB SLC-4E,0.400000,0.600000


In [9]:
#sitebar=bar_df.set_index('Launch Site',inplace=True)
#sitebar


In [24]:
fig = px.pie(spacex_df, values='class',names='Launch Site')
fig.show()

In [11]:
vbar_df=spacex_df.groupby(['Booster Version Category'])['class'].mean().reset_index()
vbar_df['failed']=1-(vbar_df['class'])
vbar_df

,Booster Version Category,class,failed
0,B4,0.545455,0.454545
1,B5,1.000000,0.000000
2,FT,0.666667,0.333333
3,v1.0,0.000000,1.000000
4,v1.1,0.066667,0.933333


In [14]:
fig = px.pie(vbar_df, values='class',names='Booster Version Category')
fig.show()

In [19]:
lc_40_df=sbar_df.loc[sbar_df['Launch Site']=='CCAFS LC-40']
lc_40_df.head()

,Launch Site,class,failed
0,CCAFS LC-40,0.269231,0.730769


In [6]:
lc_40_df=spacex_df.loc[spacex_df['Launch Site']=='CCAFS LC-40']['class'].value_counts().reset_index()
lc_40_df

,index,class
0,0,19
1,1,7


In [39]:
fig = px.pie(lc_40_df, values='class',names='class')
fig.show()

In [41]:
fig=px.scatter(spacex_df,y='class',x='Payload Mass (kg)',color='Booster Version Category')
fig

In [11]:
lc_40_df=spacex_df.loc[spacex_df['Launch Site']=='CCAFS LC-40']
lc_40_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [43]:
fig=px.scatter(lc_40_df,y='class',x='Payload Mass (kg)',color='Booster Version Category')
fig

In [19]:
pldf=spacex_df.loc[(spacex_df['Payload Mass (kg)']<7000) & (spacex_df['Payload Mass (kg)']>1000)]
pldf.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
5,5,7,CCAFS LC-40,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1


In [21]:
fig=px.scatter(pldf,y='class',x='Payload Mass (kg)',color='Booster Version Category')
fig

## The original file! don't change anything, keep it as initial status!

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server()
